In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

data_matrix = pd.read_csv('/Users/jackmayr/PycharmProjects/Notebooks/nasa.csv')
data_matrix
data_matrix = data_matrix.drop(columns = ['Miss Dist.(kilometers)', 'Orbit ID', 'Neo Reference ID', 'Name', 'Orbit Determination Date', 'Close Approach Date',  'Epoch Date Close Approach' ,'Est Dia in KM(min)', 'Est Dia in KM(max)', 'Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)', 'Est Dia in Miles(max)', 'Orbiting Body', 'Epoch Osculation','Equinox'])
data_matrix.loc[data_matrix['Hazardous']== True, 'Hazardous'] = 1 
data_matrix.loc[data_matrix['Hazardous']== False, 'Hazardous'] = 0 
data_matrix
print(len(data_matrix.columns))

24


In [8]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float32)

In [9]:
data_matrix = clean_dataset(data_matrix)
data_matrix
print(len(data_matrix[data_matrix['Hazardous'] >0]))

755


In [10]:

X = data_matrix.drop(columns = ['Hazardous'])
y = data_matrix['Hazardous']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(18, input_shape = (23,)),
                                   tf.keras.layers.Dense(32),
                                   tf.keras.layers.Dense(64),
                                   tf.keras.layers.Dense(64),
                                   tf.keras.layers.Dense(32),
                                   tf.keras.layers.Dense(16),
                                    
                                   tf.keras.layers.Dense(1, activation = 'relu')])
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, verbose = 1)
print(history.epoch, history.history['accuracy'][-1])

Epoch 1/100
118/118 [==============================] - 0s 1ms/step - loss: 2.2384 - accuracy: 0.8015
Epoch 2/100
118/118 [==============================] - 0s 1ms/step - loss: 1.8387 - accuracy: 0.8418
Epoch 3/100
118/118 [==============================] - 0s 1ms/step - loss: 2.7153 - accuracy: 0.7711
Epoch 4/100
118/118 [==============================] - 0s 1ms/step - loss: 1.9251 - accuracy: 0.8346
Epoch 5/100
118/118 [==============================] - 0s 1ms/step - loss: 1.8354 - accuracy: 0.8448
Epoch 6/100
118/118 [==============================] - 0s 1ms/step - loss: 1.5113 - accuracy: 0.8650
Epoch 7/100
118/118 [==============================] - 0s 1ms/step - loss: 1.3195 - accuracy: 0.8786
Epoch 8/100
118/118 [==============================] - 0s 1ms/step - loss: 0.7813 - accuracy: 0.9018
Epoch 9/100
118/118 [==============================] - 0s 2ms/step - loss: 0.6576 - accuracy: 0.9130
Epoch 10/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6868 - accura

In [8]:
print(model.evaluate(X_test, y_test))
#print(model.predict(X_test))
print(tf.math.confusion_matrix(y_test, model.predict(X_test)))

30/30 [==============================] - 0s 3ms/step - loss: 2.3187 - accuracy: 0.8497
[2.3186755180358887, 0.8496801853179932]
30/30 [==============================] - 0s 3ms/step
tf.Tensor(
[[797   0]
 [141   0]], shape=(2, 2), dtype=int32)
